In [49]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression,RidgeCV
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.svm import SVR
from sklearn.decomposition import *
from sklearn.metrics import mean_squared_error,make_scorer
import matplotlib.pyplot as plt

In [10]:
df = pd.read_pickle('../normalDataFromAngeloWithHDI.pickle')
df.head()

,State,HDI,awards_per_value,exp_award_value,High_School_Fee,Elementary_School_Fee,Colleges_universities_per_1000000,Junior_colleges_per_1000000,Prof_schools_per_1000000,Intensive_top_230_score,...,Property_crime,Burglary,Theft,Motor_Vehicle_theft,Per_capita_income,Price_2018_Studio,Price_2018_1br,Price_2018_2br,Price_2018_3br,Price_2018_4br
1,alabama,0.882,19.514493,63013.173913,7718.0,6388.0,8.636111,7.402381,3.289947,75.257541,...,2947.8,700.5,2006.3,241.1,38215.0,605.416667,659.000000,800.666667,1069.666667,1244.333333
2,alaska,0.942,18.414286,141431.142857,6118.0,7544.0,8.087409,1.347901,2.695803,39.089142,...,3353.0,546.3,2394.7,412.1,54430.0,814.333333,930.500000,1190.333333,1682.166667,2019.500000
3,arizona,0.912,25.563492,47830.888889,17339.0,6283.0,6.636781,6.203947,6.059670,30.154070,...,2978.4,544.4,2168.1,265.8,39955.0,701.000000,828.000000,1046.000000,1512.333333,1754.416667
4,arkansas,0.882,21.970833,51132.479167,6680.0,4698.0,8.700750,8.700750,10.708616,32.125848,...,3268.6,795.5,2233.6,239.4,39171.0,524.666667,571.166667,721.416667,977.583333,1144.833333
5,california,0.934,22.771429,63022.202857,19317.0,11363.0,6.726112,4.866240,6.089169,78.063660,...,2553.0,479.8,1623.0,450.3,54800.0,1212.333333,1426.333333,1818.916667,2519.250000,2926.333333


In [52]:
States = df.State.tolist()
X = df.drop(["State", "HDI"], axis = 1)
y = df.HDI
X.head()

,awards_per_value,exp_award_value,High_School_Fee,Elementary_School_Fee,Colleges_universities_per_1000000,Junior_colleges_per_1000000,Prof_schools_per_1000000,Intensive_top_230_score,Life_Expectancy,mc_donalds_per_100k,...,Property_crime,Burglary,Theft,Motor_Vehicle_theft,Per_capita_income,Price_2018_Studio,Price_2018_1br,Price_2018_2br,Price_2018_3br,Price_2018_4br
1,19.514493,63013.173913,7718.0,6388.0,8.636111,7.402381,3.289947,75.257541,74.813987,5.0,...,2947.8,700.5,2006.3,241.1,38215.0,605.416667,659.000000,800.666667,1069.666667,1244.333333
2,18.414286,141431.142857,6118.0,7544.0,8.087409,1.347901,2.695803,39.089142,78.915541,4.2,...,3353.0,546.3,2394.7,412.1,54430.0,814.333333,930.500000,1190.333333,1682.166667,2019.500000
3,25.563492,47830.888889,17339.0,6283.0,6.636781,6.203947,6.059670,30.154070,78.364742,4.1,...,2978.4,544.4,2168.1,265.8,39955.0,701.000000,828.000000,1046.000000,1512.333333,1754.416667
4,21.970833,51132.479167,6680.0,4698.0,8.700750,8.700750,10.708616,32.125848,75.626900,5.8,...,3268.6,795.5,2233.6,239.4,39171.0,524.666667,571.166667,721.416667,977.583333,1144.833333
5,22.771429,63022.202857,19317.0,11363.0,6.726112,4.866240,6.089169,78.063660,80.231014,3.3,...,2553.0,479.8,1623.0,450.3,54800.0,1212.333333,1426.333333,1818.916667,2519.250000,2926.333333


In [86]:
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
X_scaled = scaler_x.fit_transform(X.values)
Y_scaled = scaler_y.fit_transform(y.values.reshape(-1,1))

# Linear Regression

In [106]:
linReg = LinearRegression()
Y_pred_scaled = cross_val_predict(linReg,X_scaled,Y_scaled,cv=5)
Y_pred = scaler_y.inverse_transform(Y_pred_scaled)
print("R2 value =  {} ".format(1 - mean_squared_error(Y_pred,y)/np.std(y)))

R2 value =  0.9725698807776677 


In [108]:
linRegNorm = LinearRegression(normalize=True)
Y_pred_scaled = cross_val_predict(linRegNorm,X_scaled,Y_scaled,cv=5)
Y_pred = scaler_y.inverse_transform(Y_pred_scaled)
print("R2 value =  {}".format(1 - mean_squared_error(Y_pred,y)/np.std(y)))

R2 value =  0.9725698807776563 % 


#### With normalized inputs

### adding polynomial augmentation

In [109]:
poly = PolynomialFeatures(2)
X_poly = poly.fit_transform(X_scaled)
linReg = LinearRegression()
scores = cross_val_score(linReg,X_poly,Y_scaled,cv=5,scoring = mse_scorer)
Y_pred_scaled = cross_val_predict(linReg,X_poly,Y_scaled,cv=5)
Y_pred = scaler_y.inverse_transform(Y_pred_scaled)
print("R2 value =  {}".format(1 - mean_squared_error(Y_pred,y)/np.std(y)))

R2 value =  0.9950324662601138


In [110]:
linRegNorm = LinearRegression(normalize=True)
scores = cross_val_score(linRegNorm,X_poly,Y_scaled,cv=5,scoring = mse_scorer)
Y_pred_scaled = cross_val_predict(linRegNorm,X_poly,Y_scaled,cv=5)
Y_pred = scaler_y.inverse_transform(Y_pred_scaled)
print("R2 value =  {}".format(1 - mean_squared_error(Y_pred,y)/np.std(y)))

R2 value =  0.9755962484039037


#### With normalized inputs

So we have a tendency to overfit

# Ridge Regression

In [112]:
alphas = np.arange(0.01,10,0.01)
regr_cv = RidgeCV(alphas=alphas,fit_intercept=True,cv=5)
model_cv = regr_cv.fit(X_scaled, Y_scaled)
Y_pred_scaled = model_cv.predict(X_scaled)
Y_pred = scaler_y.inverse_transform(Y_pred_scaled)
print("R2 value =  {}".format(1 - mean_squared_error(Y_pred,y)/np.std(y)))

R2 value =  0.9987759717793506


/home/harshv834/anaconda3/envs/ada/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


#### With normalized inputs

In [116]:
regr_cv = RidgeCV(alphas=alphas,fit_intercept=True,cv=5,normalize=True)
model_cv = regr_cv.fit(X_scaled, Y_scaled)
Y_pred_scaled = model_cv.predict(X_scaled)
Y_pred = scaler_y.inverse_transform(Y_pred_scaled)
print("R2 value =  {}".format(1 - mean_squared_error(Y_pred,y)/np.std(y)))

R2 value =  0.9984426203713015


/home/harshv834/anaconda3/envs/ada/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


### adding polynomial augmentation

In [117]:
regr_cv = RidgeCV(alphas=alphas,fit_intercept=True,cv=5)
model_cv = regr_cv.fit(X_poly, Y_scaled)
Y_pred_scaled = model_cv.predict(X_poly)
Y_pred = scaler_y.inverse_transform(Y_pred_scaled)
print("R2 value =  {}".format(1 - mean_squared_error(Y_pred,y)/np.std(y)))

R2 value =  0.9996640192673864


/home/harshv834/anaconda3/envs/ada/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [129]:
regr_cv = RidgeCV(alphas=alphas,fit_intercept=True,cv=5,normalize=True)
model_cv = regr_cv.fit(X_poly, Y_scaled)
Y_pred_scaled = model_cv.predict(X_poly)
Y_pred = scaler_y.inverse_transform(Y_pred_scaled)
print("R2 value =  {}".format(1 - mean_squared_error(Y_pred,y)/np.std(y)))

R2 value =  0.9974854497806115


/home/harshv834/anaconda3/envs/ada/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


# SVR

In [130]:
parameters = {'kernel' :['rbf','linear','sigmoid','poly'],'C' : np.exp(np.arange(-6,6))}
estimator = SVR(gamma = 'auto')
clf = GridSearchCV(estimator,parameters,cv = 5,iid=True)
clf.fit(X_scaled, Y_scaled.reshape(-1,))
Y_pred_scaled = clf.predict(X_scaled).reshape(-1,1)
Y_pred = scaler_y.inverse_transform(Y_pred_scaled)
print("R2 value =  {}".format(1 - mean_squared_error(Y_pred,y)/np.std(y)))

R2 value =  0.9979270892865502
